In [1]:
import sys
sys.executable

'C:\\Users\\SEAN\\anaconda3\\envs\\Carbon_Cloud\\python.exe'

In [2]:
#Importing required libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
#Creating a dataframe
transport_df = pd.read_csv("C:\\Users\\SEAN\\Desktop\\Project\\Datasets\\CO2_Emissions_Canada.csv")
transport_df.head()

,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption City (L/100 km),Fuel Consumption Hwy (L/100 km),Fuel Consumption Comb (L/100 km),Fuel Consumption Comb (mpg),CO2 Emissions(g/km)
0,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


In [4]:
#Renaming column names
transport_df.rename(columns={'Make':'make', 'Model':'model', 'Vehicle Class':'vehicle_class', 'Engine Size(L)':'engine_size', 'Cylinders':'cylinders', 'Transmission':'transmission', 'Fuel Type':'fuel_type','Fuel Consumption City (L/100 km)':'fuel_consumption_city','Fuel Consumption Hwy (L/100 km)':'fuel_consumption_hwy','Fuel Consumption Comb (L/100 km)':'fuel_consumption_comb','Fuel Consumption Comb (mpg)':'fuel_consumption_comb_mpg','CO2 Emissions(g/km)':'co2_emissions'}, inplace=True)

#Information about the Dataframe
transport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   make                       7385 non-null   object 
 1   model                      7385 non-null   object 
 2   vehicle_class              7385 non-null   object 
 3   engine_size                7385 non-null   float64
 4   cylinders                  7385 non-null   int64  
 5   transmission               7385 non-null   object 
 6   fuel_type                  7385 non-null   object 
 7   fuel_consumption_city      7385 non-null   float64
 8   fuel_consumption_hwy       7385 non-null   float64
 9   fuel_consumption_comb      7385 non-null   float64
 10  fuel_consumption_comb_mpg  7385 non-null   int64  
 11  co2_emissions              7385 non-null   int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 692.5+ KB


In [5]:
# Qualitative / Categorical Variables :- make, model, vehicle_class, transmission, fuel_type
transport_df_qualitative = transport_df.drop(['engine_size','cylinders','fuel_consumption_city', 'fuel_consumption_hwy', 'fuel_consumption_comb', 'fuel_consumption_comb_mpg', 'co2_emissions'], axis=1)

# Quantitative Variables              :- engine_size, cylinders, fuel_consumption_city_(L/100 km), fuel_consumption_hwy_(L/100 km), fuel_consumption_comb_(L/100 km), fuel_consumption_comb_(mpg), co2_emissions(g/km)
transport_df_quantitative = transport_df.drop(['make','model', 'vehicle_class', 'transmission', 'fuel_type'], axis=1)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import f

In [7]:
# Forward Selection with only Independent Quantitative Variables
# For variable to enter the model we set the p_value (Threshold value) to 0.05

In [8]:
# IQV => Independent Quantitative Variables DV => Dependent Variable
transport_df_IQV_TO_DV = transport_df_quantitative.drop(['co2_emissions'], axis=1)

In [9]:
for i in transport_df_IQV_TO_DV.columns:
    plt.figure(figsize=(10,7))
    sns.scatterplot(data=transport_df_quantitative, x=i, y=co2_emissions'")
    plt.tight_layout()

ValueError: Could not interpret value `co2_emissions'` for `y`. An entry with this name does not appear in `data`.

<Figure size 1000x700 with 0 Axes>

In [ ]:
def create_ANOVA(formula):

    # Fit the model using the formula
    model = ols(formula, data=transport_df_quantitative).fit()
    
    # Define the sum of squares for each source of variation
    SS_reg = np.sum((model.fittedvalues - np.mean(transport_df_quantitative['co2_emissions']))**2)  # Regression sum of squares
    SS_res = np.sum(model.resid**2)  # Residual sum of squares
    SS_tot = SS_reg + SS_res  # Total sum of squares
    
    # Calculate degrees of freedom
    n_obs = len(transport_df_quantitative)  # Number of observations
    k_params = len(model.params) - 1  # Number of parameters (excluding intercept)
    df_reg = k_params  # Degrees of freedom for regression
    df_res = n_obs - k_params - 1  # Degrees of freedom for residual
    df_tot = n_obs - 1  # Total degrees of freedom
    
    # Calculate mean squares
    MS_reg = SS_reg / df_reg  # Mean square for regression
    MS_res = SS_res / df_res  # Mean square for residual
    
    # Calculate F-value
    F_value = MS_reg / MS_res
    
    # Calculate p-value
    from scipy.stats import f
    p_value = 1 - f.cdf(F_value, df_reg, df_res)
    
    # Create the ANOVA table
    anova_table_manual = pd.DataFrame({
        '': ['Regression', 'Residual', 'Total'],
        'SS': [SS_reg, SS_res, SS_tot],
        'DF': [df_reg, df_res, df_tot],
        'MS': [MS_reg, MS_res, np.nan],  # No mean square for total row
        'F': [F_value, np.nan, np.nan],  # No F-value for total row
        'Pr(>F)': [p_value, np.nan, np.nan]  # No p-value for total row
    })
    
    # Print the ANOVA table
    print(anova_table_manual)

    return SS_res, MS_res, df_res

def F_test(low_feature_SSE, SS_res, MS_res, features_added):
    f_test = ((low_feature_SSE - SS_res) / features_added) / MS_res
    print("f_test:", f_test)
    return f_test

def P_value(f_test, features_added, df_res):
    # Calculate the right-tailed F probability distribution
    probability = 1 - f.cdf(f_test, features_added, df_res)
    print("p_value:", probability)
    

In [ ]:
for i in transport_df_IQV_TO_DV.columns:
    formula = f'co2_emissions ~ {i}'
    print(formula)
    create_ANOVA(formula)
    print()

In [ ]:
# So here the selected feature is "fuel_consumption_city"

# Construct the formula string dynamically
formula = f'co2_emissions ~ fuel_consumption_city'

# Fit the model using the formula
model = ols(formula, data=transport_df_quantitative).fit()
# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
low_feature_SSE = anova_table.sum_sq['Residual']
print("low_feature_SSE : ", low_feature_SSE)
print()
print(anova_table)
print()

In [ ]:
for i in transport_df_IQV_TO_DV.columns:
    formula = f'co2_emissions ~ fuel_consumption_city + {i}'
    print(formula)
    SS_res, MS_res, df_res = create_ANOVA(formula)
    print('SSE reduction : ' , (low_feature_SSE - SS_res))
    f_test = F_test(low_feature_SSE, SS_res, MS_res, 1)
    P_value(f_test, 1, df_res)
    print()

In [ ]:
# So here the selected feature are "fuel_consumption_city + cylinders"

# Construct the formula string dynamically
formula = f'co2_emissions ~ fuel_consumption_city + cylinders'

# Fit the model using the formula
model = ols(formula, data=transport_df_quantitative).fit()
# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
low_feature_SSE = anova_table.sum_sq['Residual']
print("low_feature_SSE : ", low_feature_SSE)
print()
print(anova_table)

In [ ]:
for i in transport_df_IQV_TO_DV.columns:
    formula = f'co2_emissions ~ fuel_consumption_city + cylinders + {i}'
    print(formula)
    SS_res, MS_res, df_res = create_ANOVA(formula)
    print('SSE reduction : ' , (low_feature_SSE - SS_res))
    f_test = F_test(low_feature_SSE, SS_res, MS_res, 1)
    P_value(f_test, 1, df_res)
    print()

In [ ]:
# So here the selected feature are "fuel_consumption_city + cylinders + fuel_consumption_comb_mpg"

# Construct the formula string dynamically
formula = f'co2_emissions ~ fuel_consumption_city + cylinders + fuel_consumption_comb_mpg'

# Fit the model using the formula
model = ols(formula, data=transport_df_quantitative).fit()
# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
low_feature_SSE = anova_table.sum_sq['Residual']
print("low_feature_SSE : ", low_feature_SSE)
print()
print(anova_table)

In [ ]:
for i in transport_df_IQV_TO_DV.columns:
    formula = f'co2_emissions ~ fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + {i}'
    print(formula)
    SS_res, MS_res, df_res = create_ANOVA(formula)
    print('SSE reduction : ' , (low_feature_SSE - SS_res))
    f_test = F_test(low_feature_SSE, SS_res, MS_res, 1)
    P_value(f_test, 1, df_res)
    print()

In [ ]:
# So here the selected feature are "fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + fuel_consumption_hwy"

# Construct the formula string dynamically
formula = f'co2_emissions ~ fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + fuel_consumption_hwy'

# Fit the model using the formula
model = ols(formula, data=transport_df_quantitative).fit()
# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
low_feature_SSE = anova_table.sum_sq['Residual']
print("low_feature_SSE : ", low_feature_SSE)
print()
print(anova_table)

In [ ]:
for i in transport_df_IQV_TO_DV.columns:
    formula = f'co2_emissions ~ fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + fuel_consumption_hwy + {i}'
    print(formula)
    SS_res, MS_res, df_res = create_ANOVA(formula)
    print('SSE reduction : ' , (low_feature_SSE - SS_res))
    f_test = F_test(low_feature_SSE, SS_res, MS_res, 1)
    P_value(f_test, 1, df_res)
    print()

In [ ]:
# So here the selected feature are "fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + fuel_consumption_hwy + engine_size"

# Construct the formula string dynamically
formula = f'co2_emissions ~ fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + fuel_consumption_hwy + engine_size'

# Fit the model using the formula
model = ols(formula, data=transport_df_quantitative).fit()
# Perform ANOVA analysis
anova_table = sm.stats.anova_lm(model, typ=2)
low_feature_SSE = anova_table.sum_sq['Residual']
print("low_feature_SSE : ", low_feature_SSE)
print()
print(anova_table)

In [ ]:
for i in transport_df_IQV_TO_DV.columns:
    formula = f'co2_emissions ~ fuel_consumption_city + cylinders + fuel_consumption_comb_mpg + fuel_consumption_hwy + engine_size + {i}'
    print(formula)
    SS_res, MS_res, df_res = create_ANOVA(formula)
    print('SSE reduction : ' , (low_feature_SSE - SS_res))
    f_test = F_test(low_feature_SSE, SS_res, MS_res, 1)
    P_value(f_test, 1, df_res)
    print()

In [ ]:
# Since we will stop add here because if we add "fuel_consumption_comb" Then the p value will be 0.736 so it is greater threshold_value 0.5 so we won't be adding "fuel_consumption_comb"

In [ ]:
anova_table

In [ ]:
model.summary()